In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install preprocessor

  Created wheel for preprocessor: filename=preprocessor-1.1.3-py3-none-any.whl size=4477 sha256=3ff34fcb8a656cf2baa8aa9042cb67864dd79cc0f24a76729463eb657435107e
  Stored in directory: /root/.cache/pip/wheels/0e/b7/36/aa37256db62b4bfd35a6f1b5536e9ba843f257b79dcbf3d5f1
Successfully built preprocessor


In [4]:
!pip install tweet-preprocessor

In [5]:
import preprocessor as p

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
from preprocessor.api import clean, tokenize, parse
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import ast

#the code here is based on the reference below, after some modification and combine improvement from various other sources
#references : https://towardsdatascience.com/extracting-twitter-data-pre-processing-and-sentiment-analysis-using-python-3-0-7192bd8b47cf
#used after some modification 

#Twitter credentials for the app
consumer_key = 'WYzRl8RRj0KHBWq4GPGw17dMi'
consumer_secret = 'DjYHqBIbcNAqf0rpslQtQ82DUvn2EOMHTkHmxTFQT88r16yD9K'
access_key= '817260083587354624-Ks2DwMcBQQA8oP6p5bTiu5rrNrrtKxk'
access_secret = 'WxWI1lwTJxYR3vMVdDSbC1B7M8np5UhlAgwG7i9LXdX3T'

#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

#file location changed to "data/telemedicine_data_extraction/" for clearer path
if not os.path.exists('/content/drive/'): 
    os.mkdir('/content/drive/')
if not os.path.exists('/content/drive/MyDrive/data_covid/data/this'):
    os.mkdir('/content/drive/MyDrive/data_covid/data/this')

byu_indonesia = "/content/drive/MyDrive/data_covid/data/this/byu_indonesia.csv"

#columns of the csv file
COLS = ['id', 'created_at', 'source', 'original_text','clean_text',  'lang',
        'favorite_count', 'retweet_count', 'original_author', 'possibly_sensitive', 'hashtags',
        'user_mentions', 'place', 'place_coord_boundaries']

#set two date variables for date range
start_date = '2021-12-01'
end_date = '2022-01-02'

# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)


#mrhod clean_tweets()
def clean_tweets(tweet):
    #use slang words and stopwords to clean the data
    # the stop words and slang words that is used, comes from https://github.com/louisowen6/NLP_bahasa_resources
    # after some modification

    my_file = open("/content/drive/MyDrive/data_covid/data/combined_stop_words.txt", "r")
    content = my_file.read()
    stop_words = content.split("\n")
    file_2  = open("/content/drive/MyDrive/data_covid/data/update_combined_slang_words.txt", "r")
    content2 = file_2.read()
    slang_words = ast.literal_eval(content2)
    my_file.close()
    file_2.close()

    tweet = tweet.lower()
    #after tweepy preprocessing the colon left remain after removing mentions
    #or RT sign in the beginning of the tweet
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)

    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)

    #remove punctuation manually
    tweet = re.sub('[^a-zA-Z]', ' ', tweet)
    
    #remove tags
    tweet=re.sub("&lt;/?.*?&gt;","&lt;&gt;",tweet)
    
    #remove digits and special chars
    tweet=re.sub("(\\d|\\W)+"," ",tweet)

    #remove other symbol from tweet
    tweet = re.sub(r'â', '', tweet)
    tweet = re.sub(r'€', '', tweet)
    tweet = re.sub(r'¦', '', tweet)

    #modify the slang words into a more proper one
    word_tokens = word_tokenize(tweet)
    for w in word_tokens:
        if w in slang_words.keys():
            word_tokens[word_tokens.index(w)] = slang_words[w]

    #filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []

    #looping through conditions
    for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w.lower())
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)

#method write_tweets()
def write_tweets(keyword, file):
    # If the file exists, then read the existing data from the CSV file.
    if os.path.exists(file):
        df = pd.read_csv(file, header=0)
    else:
        df = pd.DataFrame(columns=COLS)
    #page attribute in tweepy.cursor and iteration
    for page in tweepy.Cursor(api.search, q=keyword,
                              count=200, include_rts=False, since=start_date, tweet_mode="extended").pages(100):
        for status in page:
            new_entry = []
            status = status._json

            #when run the code, below code replaces the retweet amount and
            #no of favorires that are changed since last download.
            if status['created_at'] in df['created_at'].values:
                i = df.loc[df['created_at'] == status['created_at']].index[0]
                if status['favorite_count'] != df.at[i, 'favorite_count'] or \
                   status['retweet_count'] != df.at[i, 'retweet_count']:
                    df.at[i, 'favorite_count'] = status['favorite_count']
                    df.at[i, 'retweet_count'] = status['retweet_count']
                continue

           #tweepy preprocessing called for basic preprocessing
            clean_text = clean(status['full_text'])

            #call clean_tweet method for extra preprocessing
                
            filtered_tweet=clean_tweets(clean_text)
           
            #new entry append
            new_entry += [status['id'], status['created_at'],
                          status['source'], status['full_text'],filtered_tweet,  status['lang'],
                          status['favorite_count'], status['retweet_count']]

            #to append original author of the tweet
            new_entry.append(status['user']['screen_name'])

            try:
                is_sensitive = status['possibly_sensitive']
            except KeyError:
                is_sensitive = None
            new_entry.append(is_sensitive)

            # hashtagas and mentiones are saved using comma separted
            hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
            new_entry.append(hashtags)
            mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
            new_entry.append(mentions)

            #get location of the tweet if possible
            try:
                location = status['user']['location']
            except TypeError:
                location = ''
            new_entry.append(location)

            try:
                coordinates = [coord for loc in status['place']['bounding_box']['coordinates'] for coord in loc]
            except TypeError:
                coordinates = None
            new_entry.append(coordinates)

            single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
            df = df.append(single_tweet_df, ignore_index=True)
            csvFile = open(file, 'a' ,encoding='utf-8')
    df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")

#declare keywords as a query for three categories
byu_indonesia_keywords = 'byu_id'

#call main method passing keywords and file path

write_tweets(byu_indonesia_keywords,byu_indonesia)

###Our dataset is collected, let's see how its structured

In [11]:
df = pd.read_csv('/content/drive/MyDrive/data_covid/data/this/byu_indonesia.csv')
df.head()

,id,created_at,source,original_text,clean_text,lang,favorite_count,retweet_count,original_author,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries
0,1478564382783119362,Wed Jan 05 03:09:53 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",@byu_id apa aku harus nunggu ber-jam² hingga a...,nunggu ber jam hingga aplikasi terbuka,in,0,0,rickjak16,False,NaN,byu_id,Planet Namex,NaN
1,1478561754284060674,Wed Jan 05 02:59:27 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",Min @byu_id tolong jawab lah. Jangan di skip.,min jawab skip,in,0,0,Oghee_1900,NaN,NaN,byu_id,"DKI Jakarta, Indonesia",NaN
2,1478558087564836865,Wed Jan 05 02:44:53 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",@byu_id min beberapa hari belakangan ini kok s...,min sinyal by tidak stabil hp nokia pluslokasi...,in,0,0,Andi_Pi14,NaN,NaN,byu_id,Indonesia,NaN
3,1478556956310659074,Wed Jan 05 02:40:23 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",ada\n(selalu ada buat teman)\ngokil\n(tingkah ...,teman gila tingkah gila kumpul kepomasa sih,in,0,0,AlRob1179,NaN,NaN,byu_id,"Medan Timur, Indonesia",NaN
4,1478555575197732864,Wed Jan 05 02:34:54 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",@SpeciaIONEe @worksfess @IndosatCare @triindon...,mengadopsi bebas aktif nomor hahaha,in,0,0,bangvalash,NaN,NaN,"SpeciaIONEe, worksfess, IndosatCare, triindone...","Mars, PA",NaN
